In [41]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

image = nib.load('/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/train/BraTS2021_00000/BraTS2021_00000_flair.nii.gz')
mask = nib.load('/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/train/BraTS2021_00000/BraTS2021_00000_seg.nii.gz')

image = np.asanyarray(image.dataobj, dtype=np.float32)
mask = np.asanyarray(mask.dataobj, dtype=np.float32)

image = np.pad(image, ((0, 0), (0, 0), (2, 3)), mode='constant', constant_values=0)
mask = np.pad(mask, ((0, 0), (0, 0), (2, 3)), mode='constant', constant_values=0)

print(image.shape)

image = (image - image.mean()) / image.std()

p1 = np.percentile(image, 1)
p99 = np.percentile(image, 99)
image = np.clip(image, p1, p99)

unique_vals = np.unique(image)
print(unique_vals)

'''slicez = image[150, :, :]
slicey = image[:, 150, :]
slicex = image[:, :, 100]

slices = [slicez, slicey, slicex]

fig, axes = plt.subplots(1, 3, figsize=(14, 21))
for i, s in enumerate(slices):
    axes[i].imshow(np.flipud(s.T), cmap='gray')
    axes[i].axis("off")'''

(240, 240, 160)
[-0.40676096 -0.40419534 -0.40162972 ...  3.405744    3.4083095
  3.410875  ]


'slicez = image[150, :, :]\nslicey = image[:, 150, :]\nslicex = image[:, :, 100]\n\nslices = [slicez, slicey, slicex]\n\nfig, axes = plt.subplots(1, 3, figsize=(14, 21))\nfor i, s in enumerate(slices):\n    axes[i].imshow(np.flipud(s.T), cmap=\'gray\')\n    axes[i].axis("off")'

In [1]:
import bratsdataset
from torch.utils.data import DataLoader

#constructing datasets
train_path = '/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/train/'
train_set = bratsdataset.BratsDataset(train_path)

val_path = '/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/validation/'
val_set = bratsdataset.BratsDataset(train_path)

test_path = '/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/test/'
test_set = bratsdataset.BratsDataset(train_path)

#data loaders
batch_size = 1 #8

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

#test
images, masks = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Mask batch shape:", masks.shape)

Image batch shape: torch.Size([1, 1, 240, 240, 160])
Mask batch shape: torch.Size([1, 1, 240, 240, 160])


In [ ]:
import utils
import time
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
import models

start = time.time()

model = models.Unet3Dbrats()

device = torch.device("cpu")
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1 #1000
train_loss_list = []
val_loss_list = []
early_stopping = utils.EarlyStopping(patience=10, delta=0.1)

#training loop
for epoch in range(num_epochs):

    #training
    train_loss = 0.0
    model.train()

    for images, masks in train_loader:

        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        #balance the loss per batch size
        train_loss += loss.item() * images.size(0)
    
    train_loss /= len(train_loader.dataset)

    train_loss_list.append(train_loss)

    #validation
    val_loss = 0.0
    model.eval()
    
    with torch.no_grad():

        for images, masks in val_loader:

            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)
            
            #balance the loss per batch size
            val_loss += loss.item() * images.size(0)
        
    val_loss /= len(val_loader.dataset)

    val_loss_list.append(val_loss)


    early_stopping.check(val_loss)
    if early_stopping.stop_training:
        break

    print(f"Epoch [{epoch+1}/{num_epochs}]  Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")


end = time.time()
print(f"Runtime {end - start}")

#plot training and validation loss
plt.figure()
plt.plot(train_loss_list, label='training loss')
plt.plot(val_loss_list,label='validation loss')
plt.legend()
plt.show
plt.savefig("Images/loss.png")

In [ ]:
#testing loop
test_dice = 0.0
test_loss = 0.0
model.eval()

with torch.no_grad():

    for images, masks in test_loader:

        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        dice = utils.dice_coeff(outputs, masks)
        loss = criterion(outputs, masks)
        
        #balance the loss per batch size
        test_dice += dice * images.size(0)
        test_loss += loss.item() * images.size(0)
    
test_dice /= len(test_loader.dataset)    
test_loss /= len(test_loader.dataset)

print(f"Mean dice coefficient on test set: {test_dice:.4f}")
print(f"Mean loss on test set: {test_loss:.4f}")